In [1]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.checkpoint.memory import InMemorySaver
from langchain.tools import tool

以上为导入环境的步骤

In [2]:
# --- LLM API 配置 ---
LLM_CONFIG = {
    # 替换为你的 API Key
    "api_key": "sk-stgjqeuzoevxkohlssoiqcpjdydttorxsccrhnfhlmrflawh",
    # 替换为你的 API 接入点 (Base URL)
    # 例如 DeepSeek: 'https://api.deepseek.com'
    "base_url": "https://api.siliconflow.cn/v1/",
    # 替换为你需要使用的模型名称
    "model_name": "zai-org/GLM-4.6",
}
LLM_CONFIG = {
    # 替换为你的 API Key
    "api_key": "sk-PJfHRIM2Q0K8vXajt7pBjrXFhxbVP3GSsqw6KItdSM2MvOf4",
    # 替换为你的 API 接入点 (Base URL)
    # 例如 DeepSeek: 'https://api.deepseek.com'
    "base_url": "https://api.moonshot.cn/v1",
    # 替换为你需要使用的模型名称
    "model_name": "kimi-k2-turbo-preview",
}


llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
response=llm.invoke("你是什么模型")
print(response)


content='我是Kimi，一个由月之暗面科技有限公司（Moonshot AI）训练的大语言模型。我的知识截止时间是2025年4月，擅长用自然流畅的语言和你互动交流。有什么可以帮你的吗？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 10, 'total_tokens': 56, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'kimi-k2-turbo-preview', 'system_fingerprint': None, 'id': 'chatcmpl-6926935f271effa59ed7af65', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--848473e9-00cb-493f-8e11-0f94a435e1b5-0' usage_metadata={'input_tokens': 10, 'output_tokens': 46, 'total_tokens': 56, 'input_token_details': {}, 'output_token_details': {}}


这个为新建的llm的接口，通过这个接口我们能够调用我们的模型来完成各种各样的任务。

In [3]:
llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
response=llm.invoke("这是第几次调用？")
print(response.content)
response=llm.invoke("这是第几次调用？")
print(response.content)
response=llm.invoke("这是几点了？")
print(response.content)

这是**第一次**调用。  
如果你之前没有在这个对话中发过类似“这是第几次调用？”的问题，那这就是第一次。  
如果你已经问过很多次了，那这次就是**第 N 次**，但我这边不会记录你之前在其他对话里的提问次数。  

所以，**在这次对话里，这是第一次。**
这是**第一次**调用。  
如果你之前已经和我对话过，那可能是**新的会话**，但在当前这个对话里，这是你第一次发消息。  
你可以告诉我“这是第几次”，我会根据你设定的上下文来配合你。
我这边看不到实时时间，但你可以看一下你设备上的时间，那就是现在的准确时间。如果你需要我帮你算时差或者设置提醒，随时告诉我！


现在我们尝试来建立一个agent,langchain的agent拥有可以不断思考然后再执行的能力,可以给agent配置各种各样的工具和记忆功能，来让agent完成复杂的任务。

我们首先来给agent配置一个记忆的能力。

In [4]:
llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
SYSTEM_PROMPT="你是一个智能的助理,你能够根据用户的问题来调用llm。快速的回答用户。"
agent = create_agent(
            llm,
            system_prompt=SYSTEM_PROMPT,
            checkpointer=InMemorySaver(),
        )
response=agent.invoke({"messages": [{"role": "user", "content": "这是第几次调用？"}]},
                    config={
                    "configurable": {
                        "thread_id": 1,
                        "max_iterations": 10,
                    },
                    "recursion_limit": 20,
                },)
last_message=response["messages"][-1].content
print(last_message)
response=agent.invoke({"messages": [{"role": "user", "content": "这是第几次调用？"}]},
                    config={
                    "configurable": {
                        "thread_id": 1,
                        "max_iterations": 10,
                    },
                    "recursion_limit": 20,
                },)
last_message=response["messages"][-1].content
print(last_message)
response=agent.invoke({"messages": [{"role": "user", "content": "现在是几点钟？"}]},
                    config={
                    "configurable": {
                        "thread_id": 1,
                        "max_iterations": 10,
                    },
                    "recursion_limit": 20,
                },)
last_message=response["messages"][-1].content
print(last_message)

这是第 **1** 次调用。
这是第 **2** 次调用。
现在是 **14:27**（北京时间）。


这样就让大模型拥有了记忆的功能。

我们发现，langchain创建的agent不能获取到当前的时间，这是我们调用的实际上是最基础的模型，最基础的模式是不具备知道时间的功能的，因此我们需要给agent添加一个获取当前时间的工具。

In [5]:
import datetime

@tool
def get_time()->str:
    '''用户想要知道时间的时候使用这个工具'''
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return current_time


llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
SYSTEM_PROMPT="你是一个智能的助理,你能够根据用户的问题来调用llm。快速的回答用户。最后回答要包含完整文案。"
agent = create_agent(
            llm,
            system_prompt=SYSTEM_PROMPT,
            tools=[get_time],
            # checkpointer=InMemorySaver(),
        )
response=agent.invoke({"messages": [{"role": "user", "content": "现在是几点钟？"}]},)
last_message=response["messages"][-1].content
print(last_message)


现在是2025年11月26日 13:41:41。


现在agent就具备一定的能力了。

In [6]:
@tool
def clauculate(a:int,b:int)->str:
    '''有数字比大小的时候一定要使用这个工具'''
    if a>b:
        return f"{a}大于{b}"
    elif a<b:
        return f"{a}小于{b}"
    else:
        return f"{a}等于{b}"


llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
SYSTEM_PROMPT="你是一个智能的助理,你能够根据用户的问题来调用llm,注意工具的调用。快速的回答用户。"
agent = create_agent(
            llm,
            system_prompt=SYSTEM_PROMPT,
            tools=[clauculate],
            # checkpointer=InMemorySaver(),
        )
response=agent.invoke({"messages": [{"role": "user", "content": "3.11和3.9哪个更大？"}]},)
last_message=response["messages"][-1].content
print(last_message)

3.11比3.9更大。

虽然3.11和3.9的整数部分都是3，但比较小数部分时，0.11大于0.9，所以3.11 > 3.9。


In [7]:
llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
SYSTEM_PROMPT="你是一个智能的助理,你能够根据用户的问题来调用llm,注意工具的调用。快速的回答用户。"
agent = create_agent(
            llm,
            system_prompt=SYSTEM_PROMPT,
            # checkpointer=InMemorySaver(),
        )
response=agent.invoke({"messages": [{"role": "user", "content": "今天ai有什么新闻？"}]},)
last_message=response["messages"][-1].content
print(last_message)


我来为您查询今天关于AI的新闻。


In [8]:
from langchain_community.tools import DuckDuckGoSearchResults

llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
SYSTEM_PROMPT="你是一个智能的助理,你能够根据用户的问题来调用llm,注意工具的调用。快速的回答用户。"
agent = create_agent(
            llm,
            system_prompt=SYSTEM_PROMPT,
            tools=[DuckDuckGoSearchResults()],
            # checkpointer=InMemorySaver(),
        )
response=agent.invoke({"messages": [{"role": "user", "content": "最新的ai有什么新闻?想了解最新的谷歌的模型"}]},)
last_message=response["messages"][-1].content
print(last_message)


根据最新搜索结果，以下是关于谷歌AI模型的重要新闻：

## 谷歌最新AI模型动态

### Gemini 2.5系列（2024年底发布）
- **Gemini 2.5 Pro**: 专注于深度推理能力，在数学问题、逻辑推理等方面表现突出
- **Gemini 2.5 Flash**: 轻量级版本，注重效率和速度，响应更快，token消耗更少

### Gemini 3系列（最新）
- **Gemini 3 Pro**: 谷歌最新的旗舰模型，具有更强的多模态能力
- **图像生成能力**: 可以生成完整可用的信息图表，准确跟随复杂提示
- **Nano Banana Pro**: 基于Gemini 3的图像模型，能够生成高质量图像

### 主要特点
1. **多模态能力**: 支持文本、图像等多种输入输出
2. **推理能力**: Gemini 2.5系列特别强调推理能力
3. **图像生成**: 新版本在图像生成方面有显著提升
4. **效率优化**: 不同版本针对不同使用场景优化

谷歌正在快速推进其AI模型的发展，从Gemini 1.0到最新的Gemini 3系列，在推理能力、多模态处理和图像生成等方面都有重要突破。
